# Behavior Cloning Project

### Import Libraries

In [18]:
#keras
import keras

# Helper libraries
import numpy as np
import pandas as pd
import sklearn
import math
import cv2

from sklearn.model_selection import train_test_split

import random
import csv
import matplotlib.pyplot as plt

from progressbar import *
import random

print('Keras Version: {}'.format(keras.__version__))

Keras Version: 2.2.4


### Import Data

In [19]:
def process_sample(smpl, correction_factor):
    camera_id = random.randint(0,2)
    
    if camera_id == 2:
        correction = -1*correction_factor
    else:
        correction = camera_id*correction_factor
        
    if random.random() >= 0.5:
        return [cv2.flip(cv2.imread('data/' + smpl[camera_id]), 0), (float(smpl[3])+correction)*(-1)]
    else:
        return [cv2.imread('data/' + smpl[camera_id]), float(smpl[3])+correction]

In [20]:
def generator(samples, batch_size=32, cf = 0.2):
    while True:
        random.shuffle(samples)
        for offset in range(0, len(samples), batch_size):
            subsmpls = samples[offset:offset+batch_size]
            images, measurements = zip(*[process_sample(smpl, cf) for smpl in subsmpls])
            X_train, y_train = (np.array(images), np.array(measurements))
            X_train, y_train = sklearn.utils.shuffle(X_train, y_train)
            yield (X_train, y_train)

In [21]:
reader = csv.reader(open('data/driving_log.csv'))
train_lines, valid_lines = train_test_split([line for line in reader][1:], test_size=0.2)

batch_size=48

### Define a Model

In [54]:
from keras.layers import *

def create_model():
    model = keras.Sequential()

    model.add(Lambda(lambda x: (x/255)-0.5, input_shape=(160,320,3)))
    model.add(Cropping2D(cropping=((50,20), (0,0))))

    model.add(Conv2D(32, (3, 3), activation='relu'))
    
    model.add(Dropout(0.2))
    
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(16, (3, 3), activation='relu'))
    
    model.add(Dropout(0.1))
    
    model.add(MaxPooling2D((2, 2)))

    model.add(Flatten())

    model.add(Dense(48, activation='relu'))

    model.add(Dense(1))

    model.build()

    model.compile(optimizer=keras.optimizers.Adam(), loss='mse')
    
    return model

def create_model_simple():
    model = keras.Sequential()

    model.add(Lambda(lambda x: (x/255)-0.5, input_shape=(160,320,3)))
    model.add(Cropping2D(cropping=((50,20), (0,0))))

    model.add(Flatten())

    model.add(Dense(48, activation='relu'))

    model.add(Dense(1))

    model.build()

    model.compile(optimizer=keras.optimizers.Adam(), loss='mse')
    
    return model

In [64]:
def train(train, valid, model):
    history = model.fit_generator(train, 
                steps_per_epoch=math.ceil(len(train_lines)/batch_size), 
                validation_data=valid, 
                validation_steps=math.ceil(len(valid_lines)/batch_size), 
                epochs=5, verbose=1, use_multiprocessing=True)
    return [model.evaluate_generator(valid, steps=len(valid_lines)), history]

### Save Model

In [65]:
def test_suite(model, correction_factor=0.2):
    if model is None:
        print("Specify a Model to Test")
        return
    elif not isinstance(model, list):
        if not isinstance(correction_factor, list):
            correction_factor = [correction_factor]
        mse, history = zip(*[train(generator(train_lines, batch_size = batch_size, cf = factor),
                              generator(valid_lines, batch_size = batch_size, cf = factor), model)
                             for factor in correction_factor])
    else:
        if not isinstance(correction_factor, list):
            correction_factor = [correction_factor]
        mse, history = zip(*[zip(*[train(generator(train_lines, batch_size = batch_size, cf = factor),
                              generator(valid_lines, batch_size = batch_size, cf = factor), m)
                        for factor in correction_factor]) for m in model])
    return mse, history

In [67]:
model1 = create_model_simple()
model1.summary()
model2 = create_model_simple()

correction_factors = [0.12, 0.15, 0.7] #[0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.35]

mse, history = test_suite([model1, model2], correction_factor=correction_factors)

plt.plot(correction_factors[1], mse[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_18 (Lambda)           (None, 160, 320, 3)       0         
_________________________________________________________________
cropping2d_18 (Cropping2D)   (None, 90, 320, 3)        0         
_________________________________________________________________
flatten_18 (Flatten)         (None, 86400)             0         
_________________________________________________________________
dense_35 (Dense)             (None, 48)                4147248   
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 49        
Total params: 4,147,297
Trainable params: 4,147,297
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
134/134 [==============================] - 47s 351ms/step - loss: 0.8149 - val_loss: 0.2805
Epoch 2/5
134/134 [===========

KeyboardInterrupt: 